In [1]:
import os
import csv
import MySQLdb
import EmailCreateSend
import DatabaseConnection as dbc
import LogViolation as l
pathName = os.getcwd()
index = []   #track image IDs 
logFiles = []
fileNames = os.listdir(pathName)

##Load all csv files in directory
##Script can scan multiple csv logs 
for fileNames in fileNames:
    if fileNames.endswith(".csv"):
        logFiles.append(fileNames)
        
##Loop for each log file
for i in logFiles:
    file = open(os.path.join(pathName, i), newline='')
    reader = csv.reader(file, delimiter=',')
    print('\n'+i)   ##print log file name 
    
    for row in reader:
        content = list(row[i] for i in [0,1]) ##[0,1] are included columns from CNN csv logs

        print (content)
        if float(content[1]) > .5:
            print('Violation')  
            index.append(content[0])


log-06_05_2019.csv
['1', '0.9983866214752197']
Violation
['274', '0.022467661648988724']
['275', '0.055208537727594376']
['2', '0.9958170056343079']
Violation
['3', '0.9560794830322266']
Violation
['446', '0.05130601301789284']
['1', '0.9983866214752197']
Violation
['274', '0.022467661648988724']
['275', '0.055208537727594376']
['2', '0.9958170056343079']
Violation
['3', '0.9560794830322266']
Violation
['446', '0.05130601301789284']
['1', '0.9983866214752197']
Violation
['274', '0.022467661648988724']
['275', '0.055208537727594376']
['2', '0.9958170056343079']
Violation
['3', '0.9560794830322266']
Violation
['446', '0.05130601301789284']


In [2]:
db = dbc.connect()
cursor = db.cursor()

In [3]:
#query = "SELECT U.USER_ID, U.FNAME, U.LNAME, SS.SCREENSHOT_PATH FROM USERS U LEFT JOIN SCREENSHOTS SS ON U.USER_ID = SS.SS_USER WHERE SS.SS_ID IN (%s)" % ",".join(map(str,index))
selectquery = "SELECT S.SS_USER, S.SCREENSHOT_PATH FROM SCREENSHOTS S WHERE S.image_name IN (%s)" % ",".join(map(str,index))
cursor.execute(selectquery)
results = cursor.fetchall()

In [4]:
message = ''
#EmailCreateSend.send(results)
for row in results:
    user = row[0]
    path = row[1]
    message += 'PC/USER- ' + user + ', Image Path - ' + path + '\n'
    ##LOG TO VIOLATION TABLE
    l.log(user, path)

In [5]:
print(message)

PC/USER- RAZERBLADE:Tyler, Image Path - .\dataset\single_image_test\1
PC/USER- RAZERBLADE:Tyler, Image Path - .\dataset\single_image_test\2
PC/USER- RAZERBLADE:Tyler, Image Path - .\dataset\single_image_test\3



In [6]:
EmailCreateSend.send(message)